In [1]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pprint import pprint

In [2]:
%load_ext autoreload
%autoreload 2 

project_dir = os.path.dirname(os.getcwd())
print('project direcotry is ', project_dir)
if project_dir not in sys.path:
    sys.path.append(project_dir)
print(sys.path)

project direcotry is  /home/yuan/bio/antibody-antigen-complex
['/home/yuan/anaconda3/envs/openfold-env/lib/python310.zip', '/home/yuan/anaconda3/envs/openfold-env/lib/python3.10', '/home/yuan/anaconda3/envs/openfold-env/lib/python3.10/lib-dynload', '', '/home/yuan/anaconda3/envs/openfold-env/lib/python3.10/site-packages', '/home/yuan/anaconda3/envs/openfold-env/lib/python3.10/site-packages/openfold-2.2.0-py3.10-linux-x86_64.egg', '/home/yuan/bio/antibody-antigen-complex']


## view structure

In [3]:
# get *pdb
from src.download import Download

pdb_id = '4HEP'
pdb_file = Download.download_pdb(pdb_id)

../pdb/4HEP.pdb is ready.


In [4]:
import py3Dmol
# Create a view
view = py3Dmol.view(width=400, height=300)
view.addModel(open(pdb_file).read(), 'pdb')
view.setStyle({'cartoon': {'color': 'spectrum'}})
view.zoomTo()
view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

## parse complex

In [5]:
# get chains
from src.parse_abag import ParseAbAg

outputs_dir = '../outputs'
p = ParseAbAg(pdb_file, outputs_dir)
# p.structure.header

# get chains
chains = p.get_chains()
pprint(chains)

Successfully retrieve structure of 4HEP
outputs dir:  ../outputs/4H/4HEP
[{'chains': [{'AA residues': 387,
              'chain_id': 'A',
              'seq': 'ASIKKVYRGMKNGAETINDDLEAINSELTSGGNVVHKTGDETIAGKKTFTGNVEVNGSLTLPTKSWSGELGGGIILSLRKKGTTVEYSIGGEISSSILANSNLVNRSVPNEFCPRNRCSLVGHMVGGWNAFHIDIPSSGVCQWFGPTASSGTPRGTGTYPID'},
             {'AA residues': 231,
              'chain_id': 'G',
              'seq': 'VQLVESGGGLVQPGGSLRLSCEASGFSFDDYAIGWFRQAPGKEREGVSYISMSDGRTYVADSVTGRFTISSDNAKNTVYLQMNSLKLEDTAVYYCAAGRFVTFGSAWSFVGGGPYGIDYWGKGTLVTVSS'}],
  'model_id': 0}]


In [6]:
p.structure.header['compound']

{'1': {'misc': '',
  'molecule': 'bpp',
  'chain': 'a',
  'synonym': 'baseplate protein',
  'engineered': 'yes'},
 '2': {'misc': '',
  'molecule': 'vhh17 domain',
  'chain': 'g',
  'engineered': 'yes'}}

In [7]:
# filter: is antibody-antigen complex
p.filter_antibody_antigen()
p.update_chains('chain_type')
print(p.structure_id)
pprint(p.chains[0]['chains'][0])

Pairwise chains of 4HEP are updated.
4HEP
{'AA residues': 387,
 'chain_id': 'A',
 'chain_type': 'antigen',
 'seq': 'ASIKKVYRGMKNGAETINDDLEAINSELTSGGNVVHKTGDETIAGKKTFTGNVEVNGSLTLPTKSWSGELGGGIILSLRKKGTTVEYSIGGEISSSILANSNLVNRSVPNEFCPRNRCSLVGHMVGGWNAFHIDIPSSGVCQWFGPTASSGTPRGTGTYPID'}


In [8]:
# split chains
p.chains_to_pdb()

Saved chain ['G'] to ../outputs/4H/4HEP/4HEP_G.pdb
Saved chain ['A'] to ../outputs/4H/4HEP/4HEP_A.pdb
Saved chain ['G', 'A'] to ../outputs/4H/4HEP/4HEP_G_A.pdb


In [9]:
p.bounded

[{'type': 'single-antigen',
  'chains': [['G'], ['A'], ['G', 'A']],
  'pdb_files': ['../outputs/4H/4HEP/4HEP_G.pdb',
   '../outputs/4H/4HEP/4HEP_A.pdb',
   '../outputs/4H/4HEP/4HEP_G_A.pdb']}]

## SASA

In [10]:
# calculate SASA
sasa_file = p.build_freesasa()

In [11]:
from src.analyze_sasa import AnalyzeSasa

p2= AnalyzeSasa(sasa_file)
p2.print_meta()
print(list(p2.data[0]['delta_total_sasa']))

key1 ['type', 'chains', 'pdb_files', 'sasa_files', 'delta_total_sasa']
single chain G ~ antigen chain A
------------------------------
['delta1', 'delta1_file', 'delta2', 'delta2_file']


In [12]:
# retrieve sequences
motifs = p2.retrieve_seq()
print('output file: ', motifs[1])
motifs

output file:  ../outputs/4H/4HEP/seq_sasa_delta-0_span-30.p


([('single',
   [{'seq': 'EREGVSYISMSDGRTYVADSVT',
     'sig_res': 6,
     'start': (42, '44'),
     'end': (63, '65'),
     'delta_sasa_cutoff': 0,
     'max_span': 30},
    {'seq': 'VTFGSAWSFVGGGPYG',
     'sig_res': 15,
     'start': (100, '102'),
     'end': (115, '117'),
     'delta_sasa_cutoff': 0,
     'max_span': 30}],
   'antigen',
   [{'seq': 'ASIKKVYRGMKNGAETINDDLEAINSELTSGGNVVHKTGDETIA',
     'sig_res': 20,
     'start': (0, '2'),
     'end': (43, '45'),
     'delta_sasa_cutoff': 0,
     'max_span': 30}])],
 '../outputs/4H/4HEP/seq_sasa_delta-0_span-30.p')

In [13]:
# dataset for ML
seq_sasa_file = motifs[1]
AnalyzeSasa(seq_sasa_file).build_dataset()

[{'input': 'ASIKKVYRGMKNGAETINDDLEAINSELTSGGNVVHKTGDETIA',
  'input_label': 'antigen',
  'output': 'EREGVSYISMSDGRTYVADSVT | VTFGSAWSFVGGGPYG',
  'output_label': 'single'}]

In [14]:
# complex
p2.print_delta_sasa(0)

complex: single chain G ~ antigen chain A
------------------------------
single chain:
    chain_id res_no  res aa      value
42         G     44  GLU  E   2.703881
48         G     50  TYR  Y  14.844201
57         G     59  TYR  Y  53.057039
58         G     60  VAL  V  10.669904
60         G     62  ASP  D  37.473118
63         G     65  THR  T  12.167464
100        G    102  VAL  V   9.463583
102        G    104  PHE  F  93.370081
103        G    105  GLY  G  14.680738
104        G    106  SER  S  54.540964
105        G    107  ALA  A  55.429557
106        G    108  TRP  W  35.168396
107        G    109  SER  S  51.059778
108        G    110  PHE  F  21.336189
109        G    111  VAL  V  18.810148
110        G    112  GLY  G   9.993281
111        G    113  GLY  G  60.295030
112        G    114  GLY  G  17.303302
113        G    115  PRO  P  56.590891
114        G    116  TYR  Y  51.444384
115        G    117  GLY  G   8.701156
------------------------------
antigen chain:
   chain_

## distance

In [15]:
# calculate distance
dist_file = p.build_dist()

In [16]:
from src.analyze_dist import AnalyzeDist

max_span = 30
p3= AnalyzeDist(dist_file, max_span=max_span)
p3.print_meta()
print(list(p3.data[0]['dist']))

key1 ['type', 'chains', 'pdb_files', 'dist']
single chain G ~ antigen chain A
------------------------------
['dist1', 'dist1_file', 'dist2', 'dist2_file']


In [17]:
# retrieve sequences
motifs = p3.retrieve_seq()
motifs

([('single',
   [{'seq': 'YISMSDGRTYVADSVT',
     'sig_res': 5,
     'start': (48, 50),
     'end': (63, 65),
     'dist_cutoff': 5,
     'max_span': 30},
    {'seq': 'VTFGSAWSFVGGGPYG',
     'sig_res': 15,
     'start': (100, 102),
     'end': (115, 117),
     'dist_cutoff': 5,
     'max_span': 30}],
   'antigen',
   [{'seq': 'ASIKKVYRGMKNGAETINDDLEAINSELTSGGNVVHKTGDETIA',
     'sig_res': 20,
     'start': (0, 2),
     'end': (43, 45),
     'dist_cutoff': 5,
     'max_span': 30}])],
 '../outputs/4H/4HEP/seq_dist_dist-5_span-30.p')

In [18]:
seq_dist_file = motifs[1]
AnalyzeDist(seq_dist_file).build_dataset()

[{'input': 'ASIKKVYRGMKNGAETINDDLEAINSELTSGGNVVHKTGDETIA',
  'input_label': 'antigen',
  'output': 'YISMSDGRTYVADSVT | VTFGSAWSFVGGGPYG',
  'output_label': 'single'}]

In [19]:
# complex
p3.print_min_dist(0)

------------------------------
     res_no  res atom      value aa
0         2  VAL    C  14.427739  V
1         3  GLN    C  17.190442  Q
2         4  LEU    C  15.423729  L
3         5  VAL    C  20.252827  V
4         6  GLU    C  16.777127  E
..      ...  ...  ...        ... ..
125     127  VAL    C  18.565617  V
126     128  THR    C  22.012231  T
127     129  VAL    C  20.495687  V
128     130  SER    C  23.703895  S
129     131  SER    C  24.016315  S

[130 rows x 5 columns]
------------------------------
    res_no  res atom     value aa
0        2  ALA    C  3.605043  A
1        3  SER    C  2.607683  S
2        4  ILE    C  2.718265  I
3        5  LYS    C  3.231200  K
4        6  LYS    C  4.307259  K
15      17  THR    C  4.771097  T
18      20  ASP    C  3.073532  D
19      21  ASP    C  3.869372  D
21      23  GLU    C  3.465156  E
22      24  ALA    C  3.505747  A
25      27  SER    C  3.637125  S
26      28  GLU    C  2.642103  E
29      31  SER    C  4.806912  S
30    